In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import cv2
import os
import shutil
import tqdm

cocoSource = './src/'
dataType = 'val2017'
annFile = os.path.join(cocoSource, f'annotations/instances_{dataType}.json')
cocoTarget = './target'
source_imgFolder = os.path.join(cocoSource, f'images/{dataType}')
target_imgFolder = os.path.join(cocoTarget, f'images/{dataType}')

In [ ]:
coco=COCO(annFile)

# `create_ann(img_id)`
使用方法：
* 輸入coco dataset中某一個類別的某一個影像id
* 可以產生該影像檔的yolo annotations
* 無傳回值

In [ ]:
def create_ann(img_id):

    img_info = coco.loadImgs(img_id) # 取得這張影像的資訊
    img_info = img_info[0] # 將影像資訊從{}中拿出來
    img_id = img_info['id'] # 取得影像id
    file_name = img_info['file_name'] # 取得檔案名稱，無目錄資料
    height = img_info['height'] # 取得檔案總高
    width = img_info['width'] # 取得檔案總寬

    yolo_ann_filename = os.path.join('./yolo_ann_output',os.path.splitext(file_name)[0]+".txt")
    yolo_anns = []
    annIds = coco.getAnnIds(imgIds=img_id) # 取得這個影像檔案的所有標記id
    anns = coco.loadAnns(annIds) # 將所有標記值載入陣列
    for ann in anns: # 迴圈讀所有的ann
        category_id = ann['category_id'] # 讀取ann的類別id
        if category_id not in range(1, 9): #如果不是交通工具類別，就不處理
            continue
        l1,l2,l3,l4 = ann['bbox'] # 取得bbox方框值(格式為左上x、y、長、寬)
        
        '''
        以下四行轉為yolo格式
        '''
        
        x1 = (l1 + l3 * 0.5) / width
        x2 = (l2 + l4 * 0.5) / height
        x3 = l3 / width
        x4 = l4 / height
        found_ann = f"{category_id-1} {x1} {x2} {x3} {x4}" #放入字串中
        yolo_anns.append(found_ann) # 加到list中

    if not os.path.exists(yolo_ann_filename): # 檢查檔案是否存在，不在存就建立一個，存在就跳過，表示處理過了
        with open(yolo_ann_filename, 'w') as f: 
            f.write('\n'.join(yolo_anns))  # 寫入yolo格式的annotations

# 測試一個圖檔id

In [ ]:
catId = 2 # Bicycle  
imgIds = coco.getImgIds(catIds=catId)

testimg_id = imgIds[3] # 取得這個類別的第一張影像
create_ann(testimg_id)

# 產生完整的annotations

In [ ]:
for catid in range(1,9):
    imgIds = coco.getImgIds(catIds=catid)
    for imgid in tqdm.tqdm(imgIds):
        create_ann(imgid)

# 把需要的類別挑出來複製到`target`，這邊是挑交通工具，id從1-8

In [ ]:
# def copyCat(imgId):
#     for imgid in tqdm.tqdm(imgIds):
#         img_info = coco.loadImgs(imgid)
#         source_fname = os.path.join(source_imgFolder, img_info[0]['file_name'])
#         target_fname = os.path.join(target_imgFolder, img_info[0]['file_name'])
#         shutil.copy2(source_fname, target_fname)

for catId in range(1,9):
    imgIds = coco.getImgIds(catIds=catId)
    print(catId)
    for imgid in tqdm.tqdm(imgIds):
        img_info = coco.loadImgs(imgid)
        source_fname = os.path.join(source_imgFolder, img_info[0]['file_name'])
        target_fname = os.path.join(target_imgFolder, img_info[0]['file_name'])
        shutil.copy2(source_fname, target_fname)

In [10]:
import glob
jpgfiles = glob.glob("/home/joshhu/workspace/coco_traffic/target/images/val2017/*.jpg")
with open('val.txt', 'w') as f: 
    f.write('\n'.join(jpgfiles))

In [ ]:
catId = 2 # Bicycle  
imgIds = coco.getImgIds(catIds=catId)

testimg_id = imgIds[0] # 取得這個類別的第一張影像

img_info = coco.loadImgs(testimg_id) # 取得這張影像的資訊
img_info = img_info[0] # 將影像資訊從{}中拿出來
img_id = img_info['id'] # 取得影像id
file_name = img_info['file_name'] # 取得檔案名稱，無目錄資料
height = img_info['height'] # 取得檔案總高
width = img_info['width'] # 取得檔案總寬

yolo_ann_filename = os.path.splitext(file_name)[0]+".txt"
yolo_anns = []

annIds = coco.getAnnIds(imgIds=img_id) # 取得這個影像檔案的所有標記id
anns = coco.loadAnns(annIds) # 將所有標記值載入陣列
for ann in anns:
    category_id = ann['category_id']
    if category_id not in range(1, 9):
        continue
    l1,l2,l3,l4 = ann['bbox']
    x1 = (l1 + l3 * 0.5) / width
    x2 = (l2 + l4 * 0.5) / height
    x3 = l3 / width
    x4 = l4 / height
    found_ann = f"{category_id-1} {x1} {x2} {x3} {x4}"
    yolo_anns.append(found_ann)

if not os.path.exists(yolo_ann_filename):
    with open(yolo_ann_filename, 'w') as f: 
        f.write('\n'.join(yolo_anns))


In [ ]:
f_location = os.path.join(target_imgFolder, file_name)
img = cv2.imread(f_location)
plt.imshow(img[:,:,::-1])


In [ ]:
# annIds = coco.getAnnIds(catIds=1, iscrowd=1)
# annIds = coco.getAnnIds(catIds=3)
# annIds = coco.getAnnIds(imgIds=img_info[0]['id'], iscrowd=1)